In [1]:
## Middleware
"""
Middleware in LangChain allows you to intercept and modify requests/responses
between components in your chain or agent workflow.

Common use cases:
- Logging and monitoring requests and responses
- Adding authentication or authorization checks
- Rate limiting and throttling
- Request/response transformation
- Error handling and retry logic
- Tracing and debugging

LangChain supports middleware through:
1. Callbacks - Built-in callback handlers for logging, streaming, etc.
2. Custom Runnables - Wrap components with custom logic
3. RunnableConfig - Pass configuration and callbacks through the chain
4. Listeners/Hooks - Register functions to execute at specific points

"""

'\nMiddleware in LangChain allows you to intercept and modify requests/responses\nbetween components in your chain or agent workflow.\n\nCommon use cases:\n- Logging and monitoring requests and responses\n- Adding authentication or authorization checks\n- Rate limiting and throttling\n- Request/response transformation\n- Error handling and retry logic\n- Tracing and debugging\n\nLangChain supports middleware through:\n1. Callbacks - Built-in callback handlers for logging, streaming, etc.\n2. Custom Runnables - Wrap components with custom logic\n3. RunnableConfig - Pass configuration and callbacks through the chain\n4. Listeners/Hooks - Register functions to execute at specific points\n\n'

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

#AZURE OPEN AI MODEL INTEGRATION
from langchain.chat_models import init_chat_model

#OpenAI API Key unavailable, but Azure OpenAI keys available. Hence, using Azure OpenAI model.
model = init_chat_model(
    os.getenv("AZURE_OPENAI_LLM_MODEL"),
    model_provider="azure_openai",
    deployment_name=os.getenv("AZURE_OPENAI_LLM_MODEL"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2025-01-01-preview"),
)

In [8]:
## Summarization Middleware

from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

agent = create_agent(
    model = model,
    checkpointer = InMemorySaver(),
    middleware = [ 
        SummarizationMiddleware(
            model=model,
            trigger=("messages", 10),
            keep=("messages", 5)
        ),
    ]
)

In [9]:
## Run with thread id

config = {
    "configurable" : {
        "thread_id" : "user-1234-thread"
    }
}

In [10]:
## Test Data
questions = [
    "What is 2+2?",
    "What is 10/5?",
    "What is 15 - 7?",
    "What is 3 * 4?",
    "What is the square root of 16?",
]

for q in questions:
    response = agent.invoke(
        {
            "messages": [
                SystemMessage(content="You are a helpful math tutor."),
                HumanMessage(content=q)
            ]
        },
        config=config
    )
    print(f"Messages: {response}")
    print(f"Length of Messages: {len(response['messages'])}")

Messages: {'messages': [SystemMessage(content='You are a helpful math tutor.', additional_kwargs={}, response_metadata={}, id='f46836e4-e31f-4cfd-bc89-f60e946cda5c'), HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}, id='1033b253-e783-407e-8a84-374470ad209c'), AIMessage(content='\\(2 + 2 = 4.\\)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 25, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-CxXsCR4lN95mIwVJntJdgYz2TfIq0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False

In [12]:
## Trigger based on token size

from langchain_core.tools import tool

@tool
def search_hotel(location: str) -> str:
    """Search for hotels in a given location."""
    return f"""Found hotels in {location}:
    1. Hotel A
    2. Hotel B
    3. Hotel C
    """

agent = create_agent(
    model = model,
    checkpointer = InMemorySaver(),
    middleware = [ 
        SummarizationMiddleware(
            model=model,
            trigger=("tokens", 550),
            keep=("tokens", 200),
        ),
    ]
)

In [13]:
config = {
    "configurable" : {
        "thread_id" : "user-5678-thread"
    }
}

def count_tokens(messages):
    # Simple token counting logic (for illustration purposes)
    return sum(len(msg.content.split()) for msg in messages)

In [15]:
cities = ["Paris", "New York", "Tokyo", "Sydney", "London"]

for city in cities:
    response = agent.invoke(
        {
            "messages": [
                SystemMessage(content="You are a helpful travel assistant."),
                HumanMessage(content=f"Find me hotels in {city}.")
            ]
        },
        config=config,
    )
    tokens = count_tokens(response['messages'])
    print(f"Messages: {response}")
    print(f"{city}: ~{tokens} tokens, Length of Messages: {len(response['messages'])}")

Messages: {'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nUser asked for hotel recommendations in **Paris, New York, Tokyo, Sydney, and London**.  \nAI provided curated **hotel lists by budget** (Luxury, Mid‑Range, Budget) for each city, summarizing key features (location, amenities, style).  \n\n**Tokyo:** Luxury – Peninsula, Aman, Park Hyatt, Palace Hotel; Mid‑Range – Hotel Niwa, Mitsui Garden Ginza Premier; Budget – Sakura Hotel Jimbocho, Khaosan Tokyo Origami.  \n**Sydney:** Luxury – Park Hyatt, Langham, Four Seasons, Crown Towers; Mid‑Range – Fullerton, Kimpton Margot, Ovolo Woolloomooloo, Amora Jamison; Budget – Capsule Hotel, Sydney Harbour YHA, Veriu Central, Bounce Sydney.  \n**London:**  \n- **Luxury:** Savoy, Langham, Claridge’s, Connaught, Shangri‑La The Shard.  \n- **Mid‑Range:** Hoxton Holborn, Resident Covent Garden, Treehouse, Indigo Kensington, Zetter Townhouse Clerkenwell.  \n- **Budget:** Point A Kings Cross, YHA St Pancras, Easy

In [17]:
## Fraction based trigger

summarize_model = init_chat_model(
    os.getenv("AZURE_OPENAI_LLM_MODEL"),
    model_provider="azure_openai",
    deployment_name=os.getenv("AZURE_OPENAI_LLM_MODEL"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2025-01-01-preview"),
    profile={"max_input_tokens": 400000},
)

agent = create_agent(
    model = model,
    checkpointer = InMemorySaver(),
    middleware = [ 
        SummarizationMiddleware(
            model=summarize_model,
            trigger=("fraction", 0.005),
            keep=("fraction", 0.002),
        ),
    ]
)


config = {
    "configurable" : {
        "thread_id" : "user-5678-thread"
    }
}

def count_tokens(messages):
    # Simple token counting logic (for illustration purposes)
    return sum(len(msg.content.split()) for msg in messages)

cities = ["Paris", "New York", "Tokyo", "Sydney", "London"]

for city in cities:
    response = agent.invoke(
        {
            "messages": [
                SystemMessage(content="You are a helpful travel assistant."),
                HumanMessage(content=f"Find me hotels in {city}.")
            ]
        },
        config=config,
    )
    tokens = count_tokens(response['messages'])
    fraction = tokens / 128000 #gpt-5 output token limit is 128k
    print(f"{city}: ~{fraction} fraction, Length of Messages: {len(response['messages'])}")
    print(f"Messages: {response}")

Paris: ~0.001953125 fraction, Length of Messages: 3
Messages: {'messages': [SystemMessage(content='You are a helpful travel assistant.', additional_kwargs={}, response_metadata={}, id='16ae8c15-384d-413c-a9e7-91666083dc73'), HumanMessage(content='Find me hotels in Paris.', additional_kwargs={}, response_metadata={}, id='78cbc398-6b58-41cd-b5b4-fa28c97ca9c0'), AIMessage(content='Sure! Here are some great options across different budgets and styles for hotels in Paris:\n\n---\n\n### **Luxury Hotels**\n1. **Le Meurice – Dorchester Collection (1st arrondissement)**  \n   - Location: Opposite the Tuileries Garden, near the Louvre  \n   - Features: Michelin-starred restaurant, spa, elegant decor  \n   - Ideal for: Luxury travelers, art lovers  \n\n2. **Hotel Plaza Athénée (8th arrondissement, Avenue Montaigne)**  \n   - Famous for: Eiffel Tower views, haute couture vibe  \n   - Facilities: Dior spa, multiple fine-dining options  \n\n3. **The Peninsula Paris (16th arrondissement)**  \n   - Mo

In [18]:
## Human in The Loop Middleware

from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

In [27]:
def read_email_tool(email_id: str) -> str:
    """Read the content of an email given its ID."""
    return f"Content of email {email_id}: 'Hello, this is a sample email content.'"

def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Send an email to the specified recipient."""
    return f"Email sent to {recipient} with subject '{subject}'."

agent = create_agent(
    model = model,
    tools = [read_email_tool, send_email_tool],
    checkpointer = InMemorySaver(),
    middleware = [ 
        HumanInTheLoopMiddleware(
            interrupt_on = {
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"]
                },
                "read_email_tool": False
            }
        )
    ]
)

In [28]:
config = {
    "configurable" : {
        "thread_id" : "test-approve"
    }
}

result = agent.invoke(
    {
        "messages": [
            SystemMessage(content="You are an email assistant."),
            HumanMessage(content="Send an email to jondoe@example.com with subject 'Meeting Reminder' and body 'Don't forget our meeting tomorrow at 10 AM.'")
        ]
    },
    config=config
)

In [29]:
result

{'messages': [SystemMessage(content='You are an email assistant.', additional_kwargs={}, response_metadata={}, id='2b894eda-9288-4bde-ab39-2635aff90a57'),
  HumanMessage(content="Send an email to jondoe@example.com with subject 'Meeting Reminder' and body 'Don't forget our meeting tomorrow at 10 AM.'", additional_kwargs={}, response_metadata={}, id='2d2509c5-7911-49f0-bde8-43fb3314d8f6'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 129, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-CxYFPDpfglpACySplk49Ia1KZykrr', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'f

In [30]:
from langgraph.types import Command

if "__interrupt__" in result:
    print("Human intervention required.")
    print(f"Interrupt details: {result['__interrupt__']}")

    result = agent.invoke(
        Command(
            resume={
                "decisions": [
                    {"type": "approve"}
                ]
            }
        ),
        config=config
    )

print(f"Final Result after approval: {result}")

Human intervention required.
Interrupt details: [Interrupt(value={'action_requests': [{'name': 'send_email_tool', 'args': {'recipient': 'jondoe@example.com', 'subject': 'Meeting Reminder', 'body': "Don't forget our meeting tomorrow at 10 AM."}, 'description': 'Tool execution requires approval\n\nTool: send_email_tool\nArgs: {\'recipient\': \'jondoe@example.com\', \'subject\': \'Meeting Reminder\', \'body\': "Don\'t forget our meeting tomorrow at 10 AM."}'}], 'review_configs': [{'action_name': 'send_email_tool', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='94443dda886065060134f7153b3c0a53')]
Final Result after approval: {'messages': [SystemMessage(content='You are an email assistant.', additional_kwargs={}, response_metadata={}, id='2b894eda-9288-4bde-ab39-2635aff90a57'), HumanMessage(content="Send an email to jondoe@example.com with subject 'Meeting Reminder' and body 'Don't forget our meeting tomorrow at 10 AM.'", additional_kwargs={}, response_metadata={}, id='2d2509c5-